In [1]:
import pandas as pd
import numpy as np
import random

In [4]:
apple_df = pd.read_csv('data\\apple_2011.csv',names =['date','price','percent_change'],skiprows=1)

apple_df.set_value(0,'percent_change','0')
apple_df.percent_change = apple_df.percent_change.astype('float64')
apple_df

,date,price,percent_change
0,1/3/2011,329.570,0.000000
1,1/4/2011,331.290,0.005219
2,1/5/2011,334.000,0.008180
3,1/6/2011,333.730,-0.000808
4,1/7/2011,336.120,0.007161
5,1/10/2011,342.455,0.018847
6,1/11/2011,341.640,-0.002380
7,1/12/2011,344.420,0.008137
8,1/13/2011,345.680,0.003658
9,1/14/2011,348.480,0.008100


In [8]:
import ipyparallel
from ipyparallel import Client
clients=ipyparallel.Client()
dview=clients.direct_view ()

In [12]:
%%px
import pandas as pd
import numpy as np
import random
apple_df = pd.read_csv('data\\apple_2011.csv',names =['date','price','percent_change'],skiprows=1)
apple_df.set_value(0,'percent_change','0')
apple_df.percent_change = apple_df.percent_change.astype('float64')
random.seed(7340)
last_price = apple_df['price'].loc[apple_df['price'].idxmax()]
final_df = np.array([])

count=20
final_value = 0
for j in range(0,10000):
    sample_df = pd.DataFrame(apple_df.percent_change.sample(n=count))

    sample_df = sample_df.reset_index(drop=True)
    sample_df['updated_price'] = 0.0

    sample_df.set_value(0,'updated_price',last_price + last_price*sample_df['percent_change'].loc[sample_df.index[0]])

    sample_df['percent_change']=sample_df['percent_change'] + 1

    for i in range(1,count):
        sample_df.set_value(i,'updated_price',sample_df.updated_price.iloc[[i-1]] * sample_df['percent_change'].loc[sample_df.index[i]])
    
    final_value = sample_df.get_value(count-1,'updated_price')
    final_df = np.append(final_df,final_value)

CompositeError: one or more exceptions from call to method: execute
[0:execute]: FileNotFoundError: File b'data\\apple_2011.csv' does not exist
[1:execute]: FileNotFoundError: File b'data\\apple_2011.csv' does not exist
[2:execute]: FileNotFoundError: File b'data\\apple_2011.csv' does not exist
[3:execute]: FileNotFoundError: File b'data\\apple_2011.csv' does not exist

In [5]:
final_df = np.sort(final_df)
np.percentile(final_df,1)

363.503490442356